Tous les imports dessous sont déjà dans le notebook principal

In [1]:
import os 
import torch
import numpy as np
from PIL import Image
import albumentations as A
from torch.utils.data import Dataset , DataLoader
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import sys
from pathlib import Path
import matplotlib.pyplot as plt

src_path = Path('../src').resolve()  
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from helpers import get_loaders, metrics
from plotting import visualize_results

from DeepLabv3.deeplabv3plus import Deeplabv3Plus
from DeepLabv3.dice_bce_loss import DiceBCELoss
from DeepLabv3.iou import IOU
from DeepLabv3.train_deeplab import train_deeplab

### Training `DeepLabv3` on processed data where we only keep the roofs on images.

We now want to compare another architecture named **DeepLabv3**, with UNet to leverage their different strengths. DeepLabv3 incorporates **atrous convolutions** for capturing multi-scale context and global context awareness. In the literature, it is known to be particularly good at **semantic segmentation** tasks.

Here are the **parameters** used for training the model:

In [2]:
LEARNING_RATE = 1e-4
DEVICE = "cpu" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2
NUM_EPOCHS = 1
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False

ROOT_DIR = "../data/"
images_dir = os.path.join(ROOT_DIR, "images/")
roof_images_dir = os.path.join(ROOT_DIR, "roofs/images/")
labels_dir = os.path.join(ROOT_DIR, "labels/")
weights_dir = os.path.join(ROOT_DIR, "weights/")

image_names = [file for file in os.listdir(images_dir) if file.endswith('.jpg')]

Before loading the data, we need to **standardize** the images to ensure consistent input scaling for our Convolutional Neural Network (CNN). Standardization transforms the pixel values so that they have a **mean of 0** and a **standard deviation of 1**.

In [3]:
deeplab_transform = A.Compose(
    [
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

We can now load the data, using the same segmentation dataset as before.

In [4]:
train_loader, val_loader, test_loader = get_loaders(
    image_names[:10],
    images_dir,
    labels_dir,
    BATCH_SIZE,
    deeplab_transform,
)


### Model, Loss, and Optimizer Setup

- **Model**: Using the `DeepLabv3Plus` architecture for segmentation with a single output class.
- **Loss Function**: `DiceBCELoss` to handle class imbalance and optimize segmentation performance.
- **Evaluation Metric**: Intersection over Union (`IOU`) for measuring segmentation accuracy, used before.
- **Mixed Precision**: `torch.amp.GradScaler` for efficient training on CUDA devices.
- **Optimizer**: Adam optimizer with a specified learning rate.

In [5]:
model = Deeplabv3Plus(num_classes=1).to(DEVICE)
loss_fn = DiceBCELoss()
iou_fn = IOU()
scaler = torch.amp.GradScaler("cuda")
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

C:\Users\ducou\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ducou\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


We can now train the **DeepLabv3+ model** on the initial images (i.e with no magenta), using the parameters defined above.

In [6]:
train_loss, train_iou = train_deeplab(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    loss_fn=loss_fn,
    iou_fn=iou_fn,
    optimizer=optimizer,
    device=DEVICE,
    num_epochs=NUM_EPOCHS,
    scaler=scaler,
)

Epoch: 1/1


Training Epoch 1:   0%|          | 0/4 [00:00<?, ?it/s]C:\Users\ducou\OneDrive\Bureau\Cours EPFL\MA1\Machine Learning\swiss-solar-panel-segmentation\src\segmentation_dataset.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image, mask = torch.tensor(transformed["image"]), torch.Tensor(
Training Epoch 1: 100%|██████████| 4/4 [01:53<00:00, 28.26s/it, diceloss=1.59, iou=1.15e-6]


Epoch: 1/1, Training loss: 1.703


Validation Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 3, 1000, 1000]) torch.Size([1, 1, 1000, 1000])


Validation Epoch 1: 100%|██████████| 1/1 [00:04<00:00,  4.24s/it]

tensor([0.0040, 0.0041, 0.0042,  ..., 0.6682, 0.6701, 0.6845])
Got 987064/1000000 with acc 98.71%
Dice score: 0.0136
Training complete.
